# 데이터
---
- 데이터는 대화형 데이터로 AI Hub에서 제공하는 데이터 입니다.
- 도메인은 A-I까지 9개의 도메인이 있습니다.
- A : 음식점, B : 의류, C : 학원, D : 소매점, E : 생활서비스
- F : 카페, G : 숙박업, H : 관광여가오락, I : 부동산
- 순서형 데이터로, SENTENCE 컬럼이 대화 부분 입니다.
- SPEAKERID와 SENTENCEID를 이용해 순서형 대화를 QAQ, AQA와 같이 만들어 줬습니다.
- 해당 데이터의 예시는 다음과 같습니다.

In [1]:
import pandas as pd

data = pd.read_excel("E:\dialog\D 소매점(14,949).xlsx")

In [2]:
data

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,삼겹살 1근에 얼마에요?,D,소매,정육점,1,1,가격 문의,NaN,Q,NaN,삼겹살 1근에 얼마에요?,NaN,NaN,NaN,"삼겹살, 1근",NaN,"삼겹살/부위, 1근/중량"
1,점원,만원입니다,D,소매,정육점,0,2,가격 문의,NaN,A,NaN,NaN,NaN,NaN,만원입니다,만원,NaN,만원/금액
2,고객,넷이 먹을건데 2근이면 되나요?,D,소매,정육점,1,3,0인분 용량 문의,NaN,Q,NaN,넷이 먹을 건데 2근이면 되나요?,NaN,NaN,NaN,"넷, 2근",NaN,"넷/인원, 2근/중량"
3,점원,네 충분하세요,D,소매,정육점,0,4,0인분 용량 문의,NaN,A,NaN,NaN,NaN,NaN,네 충분하세요,NaN,NaN,NaN
4,고객,그럼 2근주세요,D,소매,정육점,1,5,용량별 고기 주문,NaN,Q,NaN,그럼 2근 주세요,NaN,NaN,NaN,2근,NaN,2근/중량
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14944,고객,이 떡들 다 포장 되어있는거 맞죠?,D,소매,떡집,1,947,포장 상품 구매 확인,NaN,Q,NaN,이 떡들 다 포장 되어있는거 맞죠?,NaN,NaN,NaN,NaN,NaN,NaN
14945,고객,한 박스로도 포장 되는거죠?,D,소매,떡집,1,948,포장단위 문의,NaN,Q,NaN,한 박스로도 포장 되는거죠?,NaN,NaN,NaN,NaN,NaN,NaN
14946,고객,박스 말고 그냥 팩 단위로 포장해주실 수 있나요?,D,소매,떡집,1,949,포장단위 문의,NaN,Q,NaN,박스 말고 그냥 팩 단위로 포장해주실 수 있나요?,NaN,NaN,NaN,NaN,NaN,NaN
14947,고객,포장하는게 너무 커서 좀 적게 포장해주시면 좋을 것 같은데요?,D,소매,떡집,1,950,포장단위 문의,NaN,Q,NaN,포장하는게 너무 커서 좀 적게 포장해주시면 좋을 것 같은데요?,NaN,NaN,NaN,NaN,NaN,NaN


## 전처리된 데이터

- QAQ와 AQA는 `(back-tick) 으로 구분지었습니다.

In [3]:
with open(r"E:\dialog\all_data.txt", 'rt', encoding='utf-8') as f:
    data = f.readline()
data

'지금 배달되나요?`아 네 배달됩니다`짬뽕류는 어떤 게 있나요? 잘 나가는 짬뽕 있나요?\n'

# 데이터 로더

- 위의 data를 불러올 데이터 로더 클래스를 선언 합니다.
- 앞서 전처리 해둔 데이터를 가져옵니다.

In [4]:
from torch.utils.data import Dataset, DataLoader


class ServiceConversationDataset(Dataset):
    def __init__(self, data_path):
        self.conversation_data = []
        with open(data_path, 'rt', encoding='utf-8') as f:
            data = f.read().split("\n")
            for line in data:
                self.conversation_data.append("<s> " + line.replace("`", " <s> ") + " </s>")

    def __len__(self):
        return len(self.conversation_data)

    def __getitem__(self, idx):
        return self.conversation_data[idx]

## 디바이스 설정

In [5]:
import torch

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

## 파라미터 설정

- BATCH_SIZE : 데이터 로더의 배치 사이즈를 설정합니다.
- EPOCHS : Fine-Tuning할 Epoch을 설정합니다.
- LEARNING_RATE : 모델 학습시, lr을 설정합니다.
- WARMUP_STEPS : 스케쥴러의 warmup을 진행할 step을 설정합니다.
- DATA_PATH : Fine-Tuning에 사용할 데이터 경로를 지정합니다.
- MODEL_TYPE : 허깅페이스의 KoGPT-2를 설정합니다.
- OUTPUT_MODEL_PATH : 모델 저장 경로를 지정해 줍니다.

In [6]:
BATCH_SIZE = 4
EPOCHS = 3
LEARNING_RATE = 3e-5
WARMUP_STEPS = 100
MAX_SEQ_LEN = 100
DATA_PATH = r"E:\dialog\all_data.txt"
MODEL_TYPE = "taeminlee/kogpt2"
OUTPUT_MODEL_PATH = r"E:\online_task\service_conversation\temp"

## Pre-trained MODEL LOAD
- 허깅페이스의 KoGPT2모델 및 tokenizer를 Load합니다.

In [7]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast


model = GPT2LMHeadModel.from_pretrained(MODEL_TYPE)
tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_TYPE)

## dataset 생성 및 dataloader 생성

In [8]:
dataset = ServiceConversationDataset(DATA_PATH)
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [9]:
for d in data_loader:
    print(d)
    break

['<s> 샌드위치 다 여기서 하는 거예요? <s> 샌드위치는 저희가 수제로 만들어요 <s> 빵은 서울에서 내려오는 거예요? </s>', '<s> 시간이 없는데 빨리 나오는 메뉴는 뭔가요? <s> 파스타 종류는 거의 빨리 나옵니다 <s> 피클 좀 더 주시겠어요? </s>', '<s> 제가 혼자 해요 <s> 한 곡을 치려면 얼마나 걸리죠? <s> 취미로 하시는거에요? </s>', '<s> 블루베리 주스 한 잔 주세요 <s> 네, 블루베르 주스 한 잔 준비해드릴게요 <s> 영수증 좀 주세요 </s>']


## Fine-Tune Phase

In [10]:
from transformers import AdamW, get_linear_schedule_with_warmup


# 모델을 GPU로 보냅니다.
model = model.to(device)

# 모델을 학습하기위해 Optimizer와 scheduler를 선언 합니다.
model.train()
optimizier = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizier, WARMUP_STEPS, len(data_loader) - WARMUP_STEPS, -1)

모델 저장할 곳이 없을 경우 생성합니다.

In [11]:
import os


if not os.path.exists(OUTPUT_MODEL_PATH):
    os.mkdir(OUTPUT_MODEL_PATH)

unknown_token을 가져옵니다.

In [12]:
unknown_token = tokenizer.unk_token_id

모델 FineTuning 코드입니다.
(전처리 단계에서 미리 Q와 A를 합쳐두는 것이 좋을 것 같습니다.)


우선, 해당 코드를 실행했습니다.

In [13]:
from tqdm.notebook import tqdm


for epoch in range(EPOCHS):
    print(f'EPOCH : {epoch}, started' + "=" * 30)
    proc_seq_count = 0
    total_loss = 0.0
    total_count = 0
    temp_count = 0   # 노트북에서 보여드리기 위한 변수입니다.
    # 실제 실행시, temp_count와 관련된 코드들은 삭제해주세요.
    # 여기 부분
    with tqdm(data_loader, desc="Train Epoch #{}".format(epoch)) as t:
        for idx, data in enumerate(t):
            if temp_count > 5: # 여기
                break          # 여기
            temp_count += 1    # 여기
            train_ids = tokenizer.encode(data[0])

            for i, token in enumerate(train_ids):
                if token >= tokenizer.vocab_size:
                    train_ids[i] = unknown_token
            train_ids = torch.tensor(train_ids).to(device)
            outputs = model(train_ids, labels=train_ids)
            loss, logits = outputs[:2]
            loss.backward()
            total_loss += loss.detach().data
            total_count += 1
            t.set_postfix(loss='{:.6f}'.format(total_loss / total_count))
            optimizier.step()
            scheduler.step()
            optimizier.zero_grad()
            scheduler.optimizer.zero_grad()

            if idx % 5000 == 1:
                torch.save(model.state_dict(), os.path.join(OUTPUT_MODEL_PATH, f"KoGPT2_KoDialog_{epoch}_{idx}.pt"))
        torch.save(model.state_dict(), os.path.join(OUTPUT_MODEL_PATH, f"KoGPT2_KoDialog_{epoch}.pt"))

EPOCH : 0, started==============================



EPOCH : 1, started==============================



EPOCH : 2, started==============================


## Sentence-Generate
- FINETUNE_MODEL_PATH : 학습된 모델의 가중치 경로를 설정합니다.

In [14]:
FINETUNE_MODEL_PATH = r"E:\online_task\service_conversation\KoGPT2_KoDialog_2.pt"

QnA_Service_MODEL에 해당 가중치를 불러와 load 시켜 줍니다.

In [15]:
QnA_Service_MODEL = GPT2LMHeadModel.from_pretrained("taeminlee/kogpt2")
QnA_Service_MODEL.load_state_dict(torch.load(FINETUNE_MODEL_PATH))

<All keys matched successfully>

In [16]:
example_text = "삼겹살 1근에 얼마에요?"
encoded_text = tokenizer.encode(example_text, add_special_tokens=True, return_tensors="pt")

In [17]:
encoded_text

tensor([[26487,   106, 47667, 47442,  2288, 21888, 47774]])

In [18]:
generated_sentence = QnA_Service_MODEL.generate(encoded_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\loveg\Anaconda3\lib\site-packages\transformers\generation_utils.py:963: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


In [19]:
generated_sentence

tensor([[26487,   106, 47667, 47442,  2288, 21888, 47774, 47437,     0,   106,
         47667, 47442,   191,   209, 19991, 47437,     0, 26487, 47459,  2288]])

In [20]:
decoded_text = tokenizer.decode(generated_sentence[0], skip_special_tokens=True)

In [21]:
decoded_text

'삼겹살 1근에 얼마에요?  1근에 만 원이에요  삼겹살은 얼마'